In [1]:
import os
import sys
import pandas as pd
import numpy as np
from keras.callbacks import ModelCheckpoint
from keras.models import  Sequential
from keras.layers import  Dense, Activation
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_curve, auc
from sklearn.linear_model import Perceptron
from sklearn import preprocessing
import matplotlib.pyplot as plt

In [2]:
df = pd.read_excel(r'C:\Users\warda\Desktop\wrd\Uni\MSc\Project\C19.xlsx')
df.head()

,ID,chroniccard_mhyn,hypertension_mhyn,chronicpul_mhyn,renal_mhyn,chronicneu_mhyn,diabetes_mhyn_2,Age,ICU,Days,COVID,Sex1,Sex2,Death
0,1,0,0,0,0,0,0,38,0,12.127361,0,0,Female,0
1,2,0,0,0,0,0,0,48,0,8.702222,0,1,Male,0
2,3,0,1,0,0,0,0,54,0,11.774618,0,1,Male,0
3,4,0,0,0,0,0,0,58,0,12.670625,0,0,Female,0
4,5,0,0,1,0,0,0,31,0,11.778970,1,0,Female,0


In [3]:
df.drop(['Sex2'], axis = 1, inplace = True)
df.drop(['ID'], axis = 1, inplace = True)
df.drop(['ICU'], axis = 1, inplace = True)
df.drop(['Days'], axis = 1, inplace = True)
df.head()

,chroniccard_mhyn,hypertension_mhyn,chronicpul_mhyn,renal_mhyn,chronicneu_mhyn,diabetes_mhyn_2,Age,COVID,Sex1,Death
0,0,0,0,0,0,0,38,0,0,0
1,0,0,0,0,0,0,48,0,1,0
2,0,1,0,0,0,0,54,0,1,0
3,0,0,0,0,0,0,58,0,0,0
4,0,0,1,0,0,0,31,1,0,0


In [4]:
dataset = df.values
dataset

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 1, 0, ..., 0, 1, 0],
       ...,
       [0, 0, 0, ..., 0, 1, 0],
       [1, 1, 0, ..., 0, 1, 1],
       [0, 1, 0, ..., 1, 1, 1]], dtype=int64)

In [5]:
X = dataset[:,0:8]
Y = dataset[:,8]

In [6]:
min_max_scaler = preprocessing.MinMaxScaler()
X_scale = min_max_scaler.fit_transform(X)

In [7]:
X_scale

array([[0.        , 0.        , 0.        , ..., 0.        , 0.25641026,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.38461538,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.        , 0.46153846,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.37179487,
        0.        ],
       [1.        , 1.        , 0.        , ..., 0.        , 0.65384615,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.        , 0.78205128,
        1.        ]])

In [8]:
X_train, X_val_and_test, Y_train, Y_val_and_test = train_test_split(X_scale, Y, test_size=0.5)

In [9]:
X_val, X_test, Y_val, Y_test = train_test_split(X_val_and_test, Y_val_and_test, test_size=0.5)

In [10]:
print(X_train.shape, X_val.shape, X_test.shape, Y_train.shape, Y_val.shape, Y_test.shape)

(1118, 8) (559, 8) (560, 8) (1118,) (559,) (560,)


In [12]:
# Bringing the training data from the other split.
df2 = pd.read_csv(r'C:\Users\warda\Desktop\wrd\Uni\MSc\Project\training.csv')

In [ ]:
X_train, Y_train = 

In [ ]:
model = Sequential([
    Dense(32, activation='relu', input_shape=(8,)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid'),
])

In [ ]:
model.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
hist = model.fit(X_train, Y_train,
          batch_size=32, epochs=200,
          validation_data=(X_val, Y_val))

In [ ]:
model.evaluate(X_test, Y_test)[1]

In [ ]:
Y_pred = model.predict(X_test).ravel()

fpr, tpr, threshold = roc_curve(Y_test, Y_pred)
roc_auc = auc(fpr, tpr)

plt.rcParams["figure.figsize"] = (10.66, 10.91)
#plt.plot([-1, 0], [0, 1], color='grey', linestyle='-', alpha=0.3)
plt.plot(fpr-1, tpr, color="#e6550d",linewidth=2.2, label='Neural Network' % roc_auc)


plt.savefig("ANNROC.png", transparent=True, dpi=300)

In [ ]:
roc_auc

In [ ]:
import numpy as np
from scipy.stats import sem
from sklearn.metrics import roc_auc_score

In [ ]:
n_bootstraps = 2000
rng_seed = 42  # control reproducibility
bootstrapped_scores = []

rng = np.random.RandomState(rng_seed)
for i in range(n_bootstraps):
    indices = rng.randint(0, len(Y_pred), len(Y_pred))
    if len(np.unique(Y_test[indices])) < 2:
        continue
    score = roc_auc_score(Y_test[indices], Y_pred[indices])
    bootstrapped_scores.append(score)
    print("Bootstrap #{} ROC area: {:0.3f}".format(i + 1, score))

In [ ]:
import matplotlib.pyplot as plt
plt.hist(bootstrapped_scores, bins=50)
plt.title('Histogram of the bootstrapped ROC AUC scores')
plt.show()

In [ ]:
sorted_scores = np.array(bootstrapped_scores)
sorted_scores.sort()

confidence_lower = sorted_scores[int(0.05 * len(sorted_scores))]
confidence_upper = sorted_scores[int(0.95 * len(sorted_scores))]
print("Confidence interval : [{:0.3f} - {:0.3}]".format(
    confidence_lower, confidence_upper))

In [ ]:
Y_pred

In [ ]:
ANN = np.array(Y_pred)

In [ ]:
type(ANN)


In [ ]:
ANN = pd.DataFrame(ANN).to_csv('ANN.csv')